# Linear Dynamic Harmonic Regression (LDHR)

Para el filtrado de las series temporales (filtro de Kalman y suavizado de intervalo fijo) usaremos la toolbok [E4](https://www.ucm.es/e-4/), para ello es necesario ejecutar lo siguiente:

In [ ]:
e4init

Tambien es necesario cargar el toolbox de control

In [ ]:
pkg load control

Cargamos los datos de la serie _"lineas aéreas"_

In [ ]:
load airpas

Por comodidad vamos a generar la variable 'y' con los mismos datos (es más cómodo escribir `y` que escribir `airpas`)

In [ ]:
y = airpas;

y ahora vamos a representar los datos de la serie de líneas aéreas

In [ ]:
%plot --format png
figure(1)
grid on
hold on
title('Pasajeros de lineas aereas') % fallan los acentos
plot(airpas, 'r')

In [ ]:
#close(1) % por si generamos la figura en una ventana y la queremos cerrar después

En `PaP` guadaremos un vector con los periodos correspondientes a la estacionalidad para datos mensuales:

In [ ]:
PaP=12./(0:6)

esta serie es un tanto especial, pues el pico espectral correspondiente al periodo 2 es muy ténue (o inexistente), por ello vamos a poner dos ceros para ese componente DHR (para que no trate de identificarlo). Si no lo hacemos así, identificará un modelo para la tendencia más volatil al tratar de ajustar un componente a las oscilaciones de periodo 2 (con la serie en logaritmos si se identifica un modelo IRW para la tendencia incluso si incormporamos el componente de periodo 2).

In [ ]:
TVPaP=[1 1 1 1 1 1 0;1 0 0 0 0 0 0]

In [ ]:
[VAR,P,TVP,oar]=autodhr(y,12,[],[],PaP,TVPaP) % la primera vez que lo ejecutemos nos dará unos warnings (luego no)

Podemos calcular los ratios de varianzas (NVR) del siguiente modo:

In [ ]:
NVR=VAR(2:8)./VAR(1)

Vamos a filtrar los componentes con el modelo identificado

In [ ]:
filt=0;
[trend,season,cycle,irreg]=dhrfilt(y,P,TVP,VAR,12,filt,0);

Visualicemos los componentes

In [ ]:
plot([y,trend])

In [ ]:
plot([season])

no hemos indicado componente para un ciclo (frecuencia intermedia entre la tendencia (0) y la frecuencia del primer armónico estacional)

In [ ]:
plot(cycle) %% constante cero

In [ ]:
plot(irreg)

Pintemos la serie desestacionalizada `(trend + irreg`)

In [ ]:
APsa = trend+irreg;
plot(APsa)

In [ ]:
filt=2;  % distinta rutina de filtrado que nos devuelve cada armónico por separado
[trend2,seasonH,cycle2,irreg2]=dhrfilt(y,P,TVP,VAR,12,filt,0);

ahora `season` es una matriz cuyas columnas son los armónicos estimados

In [ ]:
plot(seasonH)

Ahora vemos la primera diferencia de la tendencia estimada:

In [ ]:
dtrend=diff(trend(:,1));
plot(dtrend)

In [ ]:
mkdir estimacionesAP

format bank

function Y = colData (cabecera,data)
    datos = num2str(data,"%5.5f");
    Y = cat(1,strjoin ({cabecera,blanks(size(datos,2)-size(cabecera,2)-1)}), datos);
endfunction

sep=repmat(' ',[size(AP,1),3]);

save("-ascii", "estimacionesAP/AP.dat",        "airpas")
save("-ascii", "estimacionesAP/APtrend.dat",   "trend")
save("-ascii", "estimacionesAP/APseason.dat",  "season")
save("-ascii", "estimacionesAP/APseasonH.dat", "seasonH")
save("-ascii", "estimacionesAP/APirreg.dat",   "irreg")
save("-ascii", "estimacionesAP/APsa.dat",      "APsa")

AP       = colData("AP",      airpas);
APtrend  = colData("APtrend",  trend);
APseason = colData("APseason",season);
APirreg  = colData("APirreg",  irreg);
APsa     = colData("APsa",      APsa);

AP_COMP_fechas=[timefmt4(12,1949,1,size(y,1)),sep, AP,sep, APtrend,sep, APseason,sep, APirreg,sep APsa];
%save("-ascii", "estimacionesAP/AP_COMP_fechas.dat",      "AP_COMP_fechas")

AP_COMP_fechas